## Setup

All libs that will be use in this note

In [1]:
import numpy as np
from glob import glob
import random

## Preprocessing

There are two folders that contains the inputs of the problem **Traveling Salesman Problem** they are different so let's split it.

### Test Data

In [2]:
PATH_TEST = 'instancias/instancias_teste/'
DATA_TEST = glob(PATH_TEST+'*.txt')
print(DATA_TEST)

['instancias/instancias_teste/ch150.txt', 'instancias/instancias_teste/bays29.txt', 'instancias/instancias_teste/berlin52.txt', 'instancias/instancias_teste/ch130.txt', 'instancias/instancias_teste/brazil58.txt', 'instancias/instancias_teste/swiss42.txt', 'instancias/instancias_teste/bayg29.txt', 'instancias/instancias_teste/bier127.txt', 'instancias/instancias_teste/descricao.txt']


### TSP Cup Data

In [3]:
PATH_CUP = 'instancias/instancias_tsp_cup/'
DATA_CUP = glob(PATH_CUP+'*.txt')
print(DATA_CUP)

['instancias/instancias_tsp_cup/tsp1.txt', 'instancias/instancias_tsp_cup/tsp2.txt', 'instancias/instancias_tsp_cup/tsp3.txt', 'instancias/instancias_tsp_cup/descricao.txt']


## Math

Some mathematics functions that will be useful 

In [4]:
def euclidian_distance(p1, p2):
    return int(np.linalg.norm(p1-p2).round())

In [5]:
print(euclidian_distance(np.array([1,1]), np.array([1,10])))

9


## Graph

We'll use a graph as data structure in order to representation our problem, where each city is a vertex and a connection is representation by a edge. 

In [6]:
class Graph:
    def __init__(self, num_vertices):
        self._adjacency_matrix = np.zeros((num_vertices, num_vertices), dtype=np.int64)
        
    def build(self, file, _type='tsp_test'):
        if _type == 'tsp_cup':
            self._handle_tsp_cup(file)
        elif _type == 'tsp_test':
            self._handle_tsp_test(file)
        else:
            print('Undefined type')
        
    def _handle_tsp_cup(self, file):
        data = []
        with open(file, 'r') as f:
            for line in f:
                try:
                    _, x, y = list(filter(lambda x: x!='', line.split(' ')))
                    data.append(np.array((float(x), float(y[:-1]))))
                except:
                    pass
        for i, p1 in enumerate(data):
            for j, p2 in enumerate(data):
                self._adjacency_matrix[i][j] = euclidian_distance(p1, p2)
    
    def _handle_tsp_test(self, file):
        data = []
        with open(file, 'r') as f:
            _input = f.readlines()
            _input = [_.strip() for _ in _input]
            _input = [_.split() for _ in _input]
            for i, array in enumerate(_input[3:]):
                try:
                    _ = [int(_) for _ in array]
                    if len(_) > 0:
                        data.append(_)
                except:
                    pass
        self._adjacency_matrix = data
    
    def num_vertices(self):
        return len(self._adjacency_matrix)
    
    def get_cost(self, _list_path_nodes):
        path = [self._adjacency_matrix[current][_next] for current, _next in zip(_list_path_nodes[1:], _list_path_nodes[:-1])]
        return sum(path)

## Optimizations Solutions

Some functions that will give us solutions for our problem.

## Helpers

Below functions that will be userful to generation solutions

In [7]:
def pick_element(_list, nodes):
    for index, x in enumerate(_list):
        if not nodes[index]:
            return index, x
    return None

In [8]:
def get_min_distance(_distances, nodes):
    min_index, min_val = pick_element(_distances, nodes)
    for index, x in enumerate(_distances):
        if min_val > x:
            min_val = x if not nodes[index] else min_val
            min_index = index if not nodes[index] else min_index
    return min_val, min_index

In [9]:
def two_opt_swap(graph, _list, i, k):
    start, middle, end = _list[:i], _list[i:k+1], _list[k+1:] + [_list[0]]
    middle.reverse()
    solution = start + middle + end
    return graph.get_cost(solution), solution

In [10]:
def shit_solution(graph, _list, i):
    solution = np.copy(_list)
    np.roll(solution, i)
    return graph.get_cost(solution), solution

In [11]:
def shake(graph, _list, i, k):
    cost, solution = shit_solution(graph, _list, i)
    return two_opt_swap(graph, solution.tolist(), i, k)

### Nearest Neighbor

In [12]:
def nearest_neighbor(graph, initial):
    solution = []
    current_node = initial
    nodes = [False for x in range(graph.num_vertices())]
    nodes[current_node] = True
    flag = True
    solution.append(current_node)
    while flag:
        distances = graph._adjacency_matrix[current_node]
        min_distance, min_index = get_min_distance(distances, nodes)
        current_node = min_index
        nodes[current_node] = True
        solution.append(current_node)
        if all(nodes):
            solution.append(initial)
            flag = False
    return graph.get_cost(solution), solution

### Random Restart

In [13]:
def random_restart(graph, initial, itermax):
    initial = np.random.randint(graph.num_vertices()) if initial is None else initial
    _iter = 0
    current_cost, current_solution = nearest_neighbor(graph, initial)
    while _iter < itermax:
        np.random.shuffle(current_solution)
        cost = graph.get_cost(current_solution)
        if current_cost > cost:
            current_cost, current_solution = cost, solution
            _iter = 0
        else:
            _iter += 1
    return graph.get_cost(current_solution), current_solution

### Best Improvement

In [14]:
def best_improvement(graph, initial):
    initial = np.random.randint(graph.num_vertices()) if initial is None else initial
    current_cost, current_solution = random_restart(graph, initial, 100)
    flag = True
    while flag:
        flag = False
        for i in range(graph.num_vertices()):
            for k in range(i+1, graph.num_vertices()):
                cost, solution = shake(graph, current_solution, i, k)
                if current_cost > cost:
                    current_cost, current_solution = cost, solution
                    flag = True
    return graph.get_cost(current_solution), current_solution

### First Improvement

In [15]:
def first_improvement(graph, initial):
    initial = np.random.randint(graph.num_vertices()) if initial is None else initial
    current_cost, current_solution = random_restart(graph, initial, 100)
    while True:
        for i in range(graph.num_vertices()):
            for k in range(i+1, graph.num_vertices()):
                cost, solution = shake(graph, current_solution, i, k)
                if current_cost > cost:
                    return current_cost, current_solution
    return graph.get_cost(current_solution), current_solution

### Variable Neighborhood Descent

In [16]:
def vnd(graph, initial, itermax):
    initial = np.random.randint(graph.num_vertices()) if initial is None else initial
    _iter = 0
    current_cost, current_solution = nearest_neighbor(graph, initial)
    while _iter < itermax:
        cost, solution = best_improvement(graph, current_solution[0])
        if current_cost > cost:
            current_cost, current_solution = cost, solution
            _iter = 0
        else:
            _iter += 1
    return graph.get_cost(current_solution), current_solution

## Test

For each test we'll build a graph with the adjacency matrix representing our cities.

### TSP TEST

In [17]:
print(DATA_TEST)

['instancias/instancias_teste/ch150.txt', 'instancias/instancias_teste/bays29.txt', 'instancias/instancias_teste/berlin52.txt', 'instancias/instancias_teste/ch130.txt', 'instancias/instancias_teste/brazil58.txt', 'instancias/instancias_teste/swiss42.txt', 'instancias/instancias_teste/bayg29.txt', 'instancias/instancias_teste/bier127.txt', 'instancias/instancias_teste/descricao.txt']


### ch150

In [18]:
graph = Graph(150)
graph.build(DATA_TEST[0], _type='tsp_test')
print(graph.num_vertices())

150


#### Nearest Neighbor

In [19]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

8073 75


#### Variable Neighborhood Descent

In [20]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 5)
print(cost, random_index)

7564 58


### bays29

In [21]:
graph = Graph(29)
graph.build(DATA_TEST[1], _type='tsp_test')
print(graph.num_vertices())

29


#### Nearest Neighbor

In [22]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

2380 10


#### Variable Neighborhood Descent

In [26]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 5)
print(cost, random_index)

2288 3


### berlin52

In [27]:
graph = Graph(52)
graph.build(DATA_TEST[2], _type='tsp_test')
print(graph.num_vertices())

52


#### Nearest Neighbor

In [28]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

9112 9


#### Variable Neighborhood Descent

In [29]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 5)
print(cost, random_index)

8848 22


### ch130

In [30]:
graph = Graph(130)
graph.build(DATA_TEST[3], _type='tsp_test')
print(graph.num_vertices())

130


#### Nearest Neighbor

In [31]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

7662 38


#### Variable Neighborhood Descent

In [32]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 5)
print(cost, random_index)

7433 36


### brazil58

In [129]:
graph = Graph(58)
graph.build(DATA_TEST[4], _type='tsp_test')
print(graph.num_vertices())

58


#### Nearest Neighbor

In [133]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

30516 11


#### Variable Neighborhood Descent

In [134]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 5)
print(cost, random_index)

27414 7


### swiss42

In [135]:
graph = Graph(42)
graph.build(DATA_TEST[5], _type='tsp_test')
print(graph.num_vertices())

42


#### Nearest Neighbor

In [139]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

1700 40


#### Variable Neighborhood Descent

In [141]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 5)
print(cost, random_index)

1547 36


### bayg29

In [143]:
graph = Graph(29)
graph.build(DATA_TEST[6], _type='tsp_test')
print(graph.num_vertices())

29


#### Nearest Neighbor

In [145]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

2091 4


#### Variable Neighborhood Descent

In [148]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 5)
print(cost, random_index)

1878 23


### bier127

In [150]:
graph = Graph(127)
graph.build(DATA_TEST[7], _type='tsp_test')
print(graph.num_vertices())

127


#### Nearest Neighbor

In [151]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

151106 68


#### Variable Neighborhood Descent

In [153]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 5)
print(cost, random_index)

139263 8


### TSP CUP

In [48]:
graph = Graph(1500)
graph.build(DATA_CUP[0], _type='tsp_cup')
print(graph._adjacency_matrix)
print(graph.num_vertices())

[[    0   203   337 ... 29025 28421 21520]
 [  203     0   400 ... 28879 28274 21365]
 [  337   400     0 ... 29269 28663 21748]
 ...
 [29025 28879 29269 ...     0   617  7700]
 [28421 28274 28663 ...   617     0  7083]
 [21520 21365 21748 ...  7700  7083     0]]
1500


In [154]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = nearest_neighbor(graph, random_index)
print(cost, random_index)

144213 13


In [155]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = random_restart(graph, random_index, 100)
print(cost, random_index)

624650 32


In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = best_improvement(graph, random_index)
print(cost, random_index)

In [156]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = first_improvement(graph, random_index)
print(cost, random_index)

618259 72


In [ ]:
random_index = np.random.randint(graph.num_vertices())
cost, solution = vnd(graph, random_index, 10)
print(cost, random_index)